<a href="https://colab.research.google.com/github/mamagoudou/QNN-with-dithering/blob/main/GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model definition

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import math

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

PATH_Models = '/content/drive/MyDrive/Memory/Models/GoogLeNet/'
PATH_Measures = '/content/drive/MyDrive/Memory/Measures/GoogLeNet/'

## Inception module

In [2]:
# adapted from 
# https://github.com/Ksuryateja/pytorch-cifar10/blob/master/models/googlenet.py
# paper: https://arxiv.org/pdf/1409.4842.pdf

class Inception(nn.Module):

  def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
    super(Inception, self).__init__()
    # 1x1 conv branch
    self.b1 = nn.Sequential(
      nn.Conv2d(in_planes, n1x1, kernel_size=1),
      nn.BatchNorm2d(n1x1),
      nn.ReLU(True),
    )

    # 1x1 conv -> 3x3 conv branch
    self.b2 = nn.Sequential(
      nn.Conv2d(in_planes, n3x3red, kernel_size=1),
      nn.BatchNorm2d(n3x3red),
      nn.ReLU(True),
      nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
      nn.BatchNorm2d(n3x3),
      nn.ReLU(True),
    )

    # 1x1 conv -> 5x5 conv branch
    self.b3 = nn.Sequential(
      nn.Conv2d(in_planes, n5x5red, kernel_size=1),
      nn.BatchNorm2d(n5x5red),
      nn.ReLU(True),
      nn.Conv2d(n5x5red, n5x5, kernel_size=3, padding=1),
      nn.BatchNorm2d(n5x5),
      nn.ReLU(True),
      nn.Conv2d(n5x5, n5x5, kernel_size=3, padding=1),
      nn.BatchNorm2d(n5x5),
      nn.ReLU(True),
    )

    # 3x3 pool -> 1x1 conv branch
    self.b4 = nn.Sequential(
      nn.MaxPool2d(3, stride=1, padding=1),
      nn.Conv2d(in_planes, pool_planes, kernel_size=1),
      nn.BatchNorm2d(pool_planes),
      nn.ReLU(True),
    )

    # Initialize weights TOTEST
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
        m.bias.data.zero_()

  def forward(self, x):
    y1 = self.b1(x)
    y2 = self.b2(x)
    y3 = self.b3(x)
    y4 = self.b4(x)
    return torch.cat([y1,y2,y3,y4], 1)

## GoogLeNet module

In [3]:
class GoogLeNet(nn.Module):

  def __init__(self):
    super(GoogLeNet, self).__init__()
    self.pre_layers = nn.Sequential(
        nn.Conv2d(3, 192, kernel_size=3, padding=1),
        nn.BatchNorm2d(192),
        nn.ReLU(True),
    )

    self.a3 = Inception(192,  64,  96, 128, 16, 32, 32)
    self.b3 = Inception(256, 128, 128, 192, 32, 96, 64)

    self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)

    self.a4 = Inception(480, 192,  96, 208, 16,  48,  64)
    self.b4 = Inception(512, 160, 112, 224, 24,  64,  64)
    self.c4 = Inception(512, 128, 128, 256, 24,  64,  64)
    self.d4 = Inception(512, 112, 144, 288, 32,  64,  64)
    self.e4 = Inception(528, 256, 160, 320, 32, 128, 128)

    self.a5 = Inception(832, 256, 160, 320, 32, 128, 128)
    self.b5 = Inception(832, 384, 192, 384, 48, 128, 128)

    self.avgpool = nn.AvgPool2d(8, stride=1)
    self.linear = nn.Linear(1024, 10)

    # Initialize weights TOTEST
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        print("test")
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
        m.bias.data.zero_()

  def forward(self, x):
    x = self.pre_layers(x)
    x = self.a3(x)
    x = self.b3(x)
    x = self.maxpool(x)
    x = self.a4(x)
    x = self.b4(x)
    x = self.c4(x)
    x = self.d4(x)
    x = self.e4(x)
    x = self.maxpool(x)
    x = self.a5(x)
    x = self.b5(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = self.linear(x)

    return x

In [4]:
# NAME_DD_MM_TEST
PATH_Name = 'GoogLeNet_15_02_TEST'

network = GoogLeNet()

epoch = 0
network.to(device)

GoogLeNet(
  (pre_layers): Sequential(
    (0): Conv2d(3, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (a3): Inception(
    (b1): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (b2): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
    (b3): Sequential(
      (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, m

# Dataset download and processing

In [5]:
import torchvision
import torchvision.transforms as transforms

In [6]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), 
                                                     (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Training & validation

In [7]:
BATCH_SIZE = 64
NUM_WORKERS = 64
MAX_EPOCHS = 50

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=NUM_WORKERS)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=NUM_WORKERS)

In [8]:
OPTIMIZER = "SGD"
LEARNING_RATE = 0.003
CRITERION = "CrossEntropyLoss"

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(network.parameters(), lr=LEARNING_RATE)

In [ ]:
import time
import csv
from tqdm.notebook import tqdm


TrainLoss = []
TrainAcc = []
Traintime = []
TestLoss = []
TestAcc = []

for epoch in tqdm(range(epoch, MAX_EPOCHS), position=0, desc="Epoch"):

  print("Epoch: %d" %(epoch))
  # TRAINING
  network.train()
  start_time = time.time()
  train_loss = 0
  correct = 0
  total = 0
  for i, data in tqdm(enumerate(trainloader, 0), position=1, desc="Training", 
                      total=len(trainloader.dataset)/BATCH_SIZE, leave=False):
    
    inputs, labels = data[0].to(device), data[1].to(device)
    optimizer.zero_grad()
    outputs = network(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()
    end_time = time.time()

  TrainLoss.append(train_loss/(i+1))
  TrainAcc.append(100.*correct/total)
  Traintime.append(end_time-start_time)
  print('TrainLoss: %.3f | TrainAcc: %.3f%% (%d/%d) | Time Elapsed %.3f sec' 
        % (TrainLoss[-1], TrainAcc[-1], correct, total, Traintime[-1]))
  
  # TESTING
  network.eval()
  test_loss = 0
  correct = 0
  total = 0
  with torch.no_grad():
    for i, data in tqdm(enumerate(testloader, 0), position=2, desc="Testing", 
                        total=len(testloader.dataset)/BATCH_SIZE, leave=False):
      inputs, labels = data[0].to(device), data[1].to(device)
      outputs = network(inputs)
      loss = criterion(outputs, labels)

      test_loss += loss.item()
      _, predicted = outputs.max(1)
      total += labels.size(0)
      correct += predicted.eq(labels).sum().item()

    TestLoss.append(test_loss/(i+1))
    TestAcc.append(100.*correct/total)
    print('TestLoss: %.3f | TestAcc: %.3f%% (%d/%d)' 
          % (TestLoss[-1], TestAcc[-1], correct, total))
    print('-' * 75)
  # SAVE MODEL IF BEST
  if TestAcc[-1] == max(TestAcc):
    torch.save({
      'optimizer': optimizer.state_dict(),
      'network': network.state_dict(),
      'epoch': epoch
    }, PATH_Models + PATH_Name + '.pth')

  if TrainAcc[-1] >= 99.9:
    break


# WRITE INFOS & STATS IN CSV
stats = {"TrainLoss": TrainLoss, "TrainAcc": TrainAcc, "Traintime": Traintime,
         "TestLoss": TestLoss, "TestAcc": TestAcc}

with open(PATH_Measures + PATH_Name + ".csv", "w") as f:
  writer = csv.writer(f)
  writer.writerow(stats.keys())
  writer.writerows(zip(*stats.values()))

infos = {"PATH_Name":PATH_Name,"BATCH_SIZE":BATCH_SIZE,"MAX_EPOCHS":MAX_EPOCHS,
         "NUM_WORKERS":NUM_WORKERS,"OPTIMIZER":OPTIMIZER,
         "LEARNING_RATE":LEARNING_RATE,"CRITERION":CRITERION,"OptimizerState":
         optimizer.state_dict(),"epoch":epoch}

with open(PATH_Measures + PATH_Name + "_infos.csv", "w") as f:
  writer = csv.DictWriter(f, fieldnames=infos.keys())
  writer.writeheader()
  writer.writerow(infos)

Epoch: 0

















TrainLoss: 2.015 | TrainAcc: 27.802% (13901/50000) | Time Elapsed 192.200 sec


TestLoss: 1.779 | TestAcc: 36.130% (3613/10000)
---------------------------------------------------------------------------
Epoch: 1


TrainLoss: 1.669 | TrainAcc: 39.558% (19779/50000) | Time Elapsed 202.280 sec


TestLoss: 1.565 | TestAcc: 43.470% (4347/10000)
---------------------------------------------------------------------------
Epoch: 2


...

TestLoss: 1.172 | TestAcc: 69.730% (6973/10000)
---------------------------------------------------------------------------
Epoch: 37


TrainLoss: 0.055 | TrainAcc: 98.990% (49495/50000) | Time Elapsed 206.203 sec


TestLoss: 1.191 | TestAcc: 69.830% (6983/10000)
---------------------------------------------------------------------------
Epoch: 38


TrainLoss: 0.048 | TrainAcc: 99.208% (49604/50000) | Time Elapsed 204.467 sec


TestLoss: 1.170 | TestAcc: 70.590% (7059/10000)
---------------------------------------------------------------------------
Epoch: 39


In [ ]:
# WRITE INFOS & STATS IN CSV
stats = {"TrainLoss": TrainLoss, "TrainAcc": TrainAcc, "Traintime": Traintime,
         "TestLoss": TestLoss, "TestAcc": TestAcc}

with open(PATH_Measures + PATH_Name + ".csv", "w") as f:
  writer = csv.writer(f)
  writer.writerow(stats.keys())
  writer.writerows(zip(*stats.values()))

infos = {"PATH_Name":PATH_Name,"BATCH_SIZE":BATCH_SIZE,"MAX_EPOCHS":MAX_EPOCHS,
         "NUM_WORKERS":NUM_WORKERS,"OPTIMIZER":OPTIMIZER,
         "LEARNING_RATE":LEARNING_RATE,"CRITERION":CRITERION,"OptimizerState":
         optimizer.state_dict(),"epoch":epoch}

with open(PATH_Measures + PATH_Name + "_infos.csv", "w") as f:
  writer = csv.DictWriter(f, fieldnames=infos.keys())
  writer.writeheader()
  writer.writerow(infos)

# Drafts

In [ ]:
# test inspiration: https://github.com/Ksuryateja/pytorch-cifar10/blob/master/cifar10.py